In [1]:
cd ~/ibm_lg/kmer-hashing/brawand2011/nfcore-rnaseq/human/single_end/STAR 

/mnt/ibm_lg/olga/kmer-hashing/brawand2011/nfcore-rnaseq/human/single_end/STAR


In [2]:
sample_id = 'SRR306838_GSM752691_hsa_br_F_1'

In [3]:
ls -lha

total 21G
drwxr-xr-x  3 5008 5000 4.0K Sep 10 12:25 ./
drwxr-xr-x 17 5008 5000 4.0K Sep  2 12:00 ../
drwxr-xr-x  2 5008 5000 8.0K Sep 10 12:25 logs/
-rw-r--r--  1 5008 5000  65M Sep 10 10:02 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam
-rw-r--r--  1 5008 5000 2.5M Sep 10 10:02 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.bai
-rw-r--r--  1 5008 5000 1.3G Sep 10 10:42 SRR306839_GSM752692_hsa_br_M_3Aligned.sortedByCoord.out.bam
-rw-r--r--  1 5008 5000 3.6M Sep 10 10:42 SRR306839_GSM752692_hsa_br_M_3Aligned.sortedByCoord.out.bam.bai
-rw-r--r--  1 5008 5000 1.5G Sep 10 10:55 SRR306841_GSM752694_hsa_br_M_2Aligned.sortedByCoord.out.bam
-rw-r--r--  1 5008 5000 3.8M Sep 10 10:55 SRR306841_GSM752694_hsa_br_M_2Aligned.sortedByCoord.out.bam.bai
-rw-r--r--  1 5008 5000 405M Sep 10 09:25 SRR306843_GSM752696_hsa_br_M_5Aligned.sortedByCoord.out.bam
-rw-r--r--  1 5008 5000 3.5M Sep 10 09:25 SRR306843_GSM752696_hsa_br_M_5Aligned.sortedByCoord.out.bam.bai
-rw-r--r--  1 5008 

#### Filter GTF for only CDS sequences

In [6]:
gtf = '/home/olga/data_lg/czbiohub-reference/gencode/human/v30/gencode.v30.annotation.ERCC92.gtf'
! wc -l $gtf

cds_gtf = '/home/olga/data_lg/czbiohub-reference/gencode/human/v30/gencode.v30.annotation.ERCC92.CDS.gtf'
! bioawk -c gff '$feature == "CDS"' $gtf > $cds_gtf

! wc -l $cds_gtf

2774396 /home/olga/data_lg/czbiohub-reference/gencode/human/v30/gencode.v30.annotation.ERCC92.gtf
753983 /home/olga/data_lg/czbiohub-reference/gencode/human/v30/gencode.v30.annotation.ERCC92.CDS.gtf


In [18]:


noncoding_exon_gtf = '/home/olga/data_lg/czbiohub-reference/gencode/human/v30/gencode.v30.annotation.ERCC92.exon.gtf'
! bioawk -c gff '$feature == "exon"' $gtf > $exon_gtf

! wc -l $cds_gtf

753983 /home/olga/data_lg/czbiohub-reference/gencode/human/v30/gencode.v30.annotation.ERCC92.CDS.gtf


In [19]:


utr_gtf = '/home/olga/data_lg/czbiohub-reference/gencode/human/v30/gencode.v30.annotation.ERCC92.UTR.gtf'
! bioawk -c gff '$feature == "UTR"' $gtf > $utr_gtf

! wc -l $cds_gtf

753983 /home/olga/data_lg/czbiohub-reference/gencode/human/v30/gencode.v30.annotation.ERCC92.CDS.gtf


What are the feature types?

In [7]:
! cut -f 3 $gtf | sort | uniq

CDS
##contact: gencode-help@ebi.ac.uk
##date: 2019-03-28
##description: evidence-based annotation of the human genome (GRCh38), version 30 (Ensembl 96)
exon
##format: gtf
gene
##provider: GENCODE
Selenocysteine
start_codon
stop_codon
transcript
UTR


#### Filter for only stop codons


In [10]:
stop_codon_gtf = '/home/olga/data_lg/czbiohub-reference/gencode/human/v30/gencode.v30.annotation.ERCC92.stop_codon.gtf'
! bioawk -c gff '$feature == "stop_codon"' $gtf > $stop_codon_gtf
! wc -l $stop_codon_gtf

79378 /home/olga/data_lg/czbiohub-reference/gencode/human/v30/gencode.v30.annotation.ERCC92.stop_codon.gtf


In [ ]:
! bedtools intersect $gtf

#### Remove CDS sequences overlapping stop codons

In [9]:
cds_no_stop_codons_gtf = '/home/olga/data_lg/czbiohub-reference/gencode/human/v30/gencode.v30.annotation.ERCC92.CDS_without_stop_codon.gtf'


! bedtools subtract -a $cds_gtf -b $stop_codon_gtf > $cds_no_stop_codons_gtf
! wc -l $cds_no_stop_codons_gtf

798376 /home/olga/data_lg/czbiohub-reference/gencode/human/v30/gencode.v30.annotation.ERCC92.CDS_without_stop_codon.gtf


#### Remove CDS sequences overlapping stop codons or UTRs

In [21]:
cds_no_stop_codons_or_utr_gtf = '/home/olga/data_lg/czbiohub-reference/gencode/human/v30/gencode.v30.annotation.ERCC92.CDS_without_stop_codon_or_utr.gtf'

# -A flag to remove the whole feature in -a if it overlaps at all with -b
! bedtools subtract -A -a $cds_gtf -b $stop_codon_gtf | bedtools subtract -A -a - -b $utr_gtf > $cds_no_stop_codons_or_utr_gtf
! wc -l $cds_no_stop_codons_or_utr_gtf

482824 /home/olga/data_lg/czbiohub-reference/gencode/human/v30/gencode.v30.annotation.ERCC92.CDS_without_stop_codon_or_utr.gtf


In [37]:
! head -n 20 $cds_no_stop_codons_or_utr_gtf

chr1	HAVANA	CDS	65565	65573	.	+	0	gene_id "ENSG00000186092.6"; transcript_id "ENST00000641515.2"; gene_type "protein_coding"; gene_name "OR4F5"; transcript_type "protein_coding"; transcript_name "OR4F5-202"; exon_number 2; exon_id "ENSE00003813641.1"; level 2; protein_id "ENSP00000493376.2"; tag "RNA_Seq_supported_partial"; tag "basic"; havana_gene "OTTHUMG00000001094.4"; havana_transcript "OTTHUMT00000003223.4";
chr1	ENSEMBL	CDS	69091	70005	.	+	0	gene_id "ENSG00000186092.6"; transcript_id "ENST00000335137.4"; gene_type "protein_coding"; gene_name "OR4F5"; transcript_type "protein_coding"; transcript_name "OR4F5-201"; exon_number 1; exon_id "ENSE00002319515.2"; level 3; protein_id "ENSP00000334393.3"; transcript_support_level "NA"; tag "basic"; tag "appris_principal_1"; tag "CCDS"; ccdsid "CCDS30547.1"; havana_gene "OTTHUMG00000001094.4";
chr1	HAVANA	CDS	450743	451678	.	-	0	gene_id "ENSG00000284733.1"; transcript_id "ENST00000426406.3"; gene_type "protein_coding"; gene_name "OR4F29"; t

### Filter BAM file for only reads overlapping coding sequences

In [90]:
bam = f'{sample_id}Aligned.sortedByCoord.out.bam'
coding_bam = f'{sample_id}Aligned.sortedByCoord.out_CDS_without_stop_codon_or_utr.bam'
outdir = '/home/olga/data_sm/kmer-hashing/brawand2011/human'

!mkdir -p $outdir

### Filter for only unspliced reads to make my life easier

In [91]:
pwd

'/mnt/ibm_lg/olga/kmer-hashing/brawand2011/nfcore-rnaseq/human/single_end/STAR'

In [92]:
bam

'SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam'

In [93]:
! samtools view -h -F 4 $bam | awk '$6 !~ /N/ || $1 ~ /@/' | samtools view -bS - > $outdir/$bam\.unspliced.bam

In [95]:
! samtools view $outdir/SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.bam | head

SRR306838.2504624	256	chr1	14362	1	36M3D27M	*	0	0	TCCTGCACAGCTAGAGATCCTTTATTAAAAGCACACTGGTTTCTGCTCAGTTCTTTATTGATT	-262773:5384472767787875666655876446966448:86379453866:78998886	NH:i:4	HI:i:2	AS:i:53	nM:i:0	RG:Z:SRR306838_GSM752691_hsa_br_F_1
SRR306838.13797218	256	chr1	14978	0	52M	*	0	0	CTCATGACCAGCTTGTTGAAGAGATCCGACATCAAGTGCCCACCTTGGCTCG	3254352877855542774474744666666466364788766764655485	NH:i:10	HI:i:2	AS:i:51	nM:i:0	RG:Z:SRR306838_GSM752691_hsa_br_F_1
SRR306838.12718178	256	chr1	14985	0	52M	*	0	0	CCAGCTTGTTGAAGAGATCCGACATCAAGTCCCCACCTTGGCTCGTGGCTCT	414713:668526626656774656765848876456445676864564475	NH:i:10	HI:i:2	AS:i:49	nM:i:1	RG:Z:SRR306838_GSM752691_hsa_br_F_1
SRR306838.7979691	256	chr1	17598	0	64M	*	0	0	GGCCATACAGGTTCTCGGTGGTGTTGAAGAGCAGCAAGGAGCTGACAGAGCTGATGTTGCTGGG	2535633843737:5567588675884585756865375385783445356:777658756678	NH:i:5	HI:i:3	AS:i:62	nM:i:0	RG:Z:SRR306838_GSM752691_hsa_br_F_1
SRR306838.11927233	256	chr1	18229	0	68M	*	0	0	CTGCCCAGTAGCCTCAGGACACAGGGGTATGGGGA

In [94]:
! samtools view $outdir/$bam\.unspliced.bam | wc -l

807698


In [101]:
ls -lha $outdir

total 331M
drwxrwxr-x 2 olga olga 4.0K Sep 18 09:59 ./
drwxrwxr-x 3 olga olga 4.0K Sep 10 15:07 ../
-rw-rw-r-- 1 olga olga  52M Sep 18 10:05 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.bam
-rw-rw-r-- 1 olga olga 2.9M Sep 18 09:59 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.bam.bai
-rw-rw-r-- 1 olga olga    0 Sep 18 10:10 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.out_CDS_without_stop_codon_or_utr.bam
-rw-rw-r-- 1 olga olga  15M Sep 10 15:55 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codon.bam
-rw-rw-r-- 1 olga olga 1.7M Sep 18 07:05 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codon.bam.bai
-rw-rw-r-- 1 olga olga  15M Sep 18 07:15 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codon_or_utr.bam
-rw-rw-r-- 1 olga olga 1.7M Sep 18 07:06 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codon_or_utr.bam.bai

In [102]:
! time bedtools intersect -a $outdir/$bam\.unspliced.bam -b $cds_no_stop_codons_or_utr_gtf > $outdir/$bam\.unspliced.out_CDS_without_stop_codon_or_utr.bam

***** WARNING: File /home/olga/data_sm/kmer-hashing/brawand2011/human/SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.bam has inconsistent naming convention for record:
GL000008.2	29409	29480	SRR306838.11378224	0	+

***** WARNING: File /home/olga/data_sm/kmer-hashing/brawand2011/human/SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.bam has inconsistent naming convention for record:
GL000008.2	29409	29480	SRR306838.11378224	0	+

2.82user 32.63system 0:35.56elapsed 99%CPU (0avgtext+0avgdata 450136maxresident)k
0inputs+19008outputs (0major+242578minor)pagefaults 0swaps


In [103]:
! samtools view $outdir/$bam\.unspliced.out_CDS_without_stop_codon_or_utr.bam | head

SRR306838.13598607	0	chr1	945525	255	69M	*	0	0	CGAGAATCCCTCGGTGTCGTCCTCTTCAGAGCTGTTCAGGTCAAAGAGGTCTTTAAATTGCTTCCTGTC	33273589652656241634:568456485768435736637444656539776445766784767539	NH:i:1	HI:i:1	AS:i:67	nM:i:0	RG:Z:SRR306838_GSM752691_hsa_br_F_1
SRR306838.23303318	0	chr1	945534	255	70M	*	0	0	CTCGGTGTCGTCCTCTTCAGAGCTGTTCAGGTCAAAGAGGTCTTTAAATTGCTTCCTGTCCTCATCCTTC	32255/54320856423955475855243842544364753684733245755564853:5796595628	NH:i:1	HI:i:1	AS:i:68	nM:i:0	RG:Z:SRR306838_GSM752691_hsa_br_F_1
SRR306838.19917713	0	chr1	945543	255	69M	*	0	0	GTCCTCTTCAGAGCTGTTCAGGTCAAAGAGGTCTTTAAATTGCTTCCTGTCCTCATCCTTCCTGTCAGC	013212745640534433447437630547327254334675548668448538849374:67647685	NH:i:1	HI:i:1	AS:i:67	nM:i:0	RG:Z:SRR306838_GSM752691_hsa_br_F_1
SRR306838.19937719	0	chr1	945546	255	70M	*	0	0	CTCTTCAGAGCTGTTCAGGTCAAAAAGGTCTTTAAATTGCTTCCTGTCCTCATCCTTCCTGTCAGCCATC	222/141512657236476374544365/655564455565677865:4473365546686585:55757	NH:i:1	HI:i:1	AS:i:66	nM:i:1	RG:Z:SRR306838_GSM752691

In [82]:
ls -lha $outdir

total 337M
drwxrwxr-x 2 olga olga 4.0K Sep 18 07:37 ./
drwxrwxr-x 3 olga olga 4.0K Sep 10 15:07 ../
-rw-rw-r-- 1 olga olga  52M Sep 18 07:37 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.bam
-rw-rw-r-- 1 olga olga 9.3M Sep 18 09:58 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.out_CDS_without_stop_codon_or_utr.bam
-rw-rw-r-- 1 olga olga  15M Sep 10 15:55 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codon.bam
-rw-rw-r-- 1 olga olga 1.7M Sep 18 07:05 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codon.bam.bai
-rw-rw-r-- 1 olga olga  15M Sep 18 07:15 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codon_or_utr.bam
-rw-rw-r-- 1 olga olga 1.7M Sep 18 07:06 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codon_or_utr.bam.bai
-rw-rw-r-- 1 olga olga 4.1M Sep 18 07:17 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codo

## Index all sam files

In [104]:
! samtools index $outdir/SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.bam

In [89]:
outdir

'/home/olga/data_sm/kmer-hashing/brawand2011/human/'

In [108]:
! samtools index $outdir/SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.out_CDS_without_stop_codon_or_utr.bam

In [109]:
ls -lha $outdir

total 342M
drwxrwxr-x 2 olga olga 4.0K Sep 18 10:12 ./
drwxrwxr-x 3 olga olga 4.0K Sep 10 15:07 ../
-rw-rw-r-- 1 olga olga  52M Sep 18 10:05 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.bam
-rw-rw-r-- 1 olga olga 2.9M Sep 18 10:12 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.bam.bai
-rw-rw-r-- 1 olga olga 9.3M Sep 18 10:12 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.out_CDS_without_stop_codon_or_utr.bam
-rw-rw-r-- 1 olga olga 1.9M Sep 18 10:12 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.out_CDS_without_stop_codon_or_utr.bam.bai
-rw-rw-r-- 1 olga olga  15M Sep 10 15:55 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codon.bam
-rw-rw-r-- 1 olga olga 1.7M Sep 18 07:05 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codon.bam.bai
-rw-rw-r-- 1 olga olga  15M Sep 18 07:15 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_sto

In [87]:
ls -lha $outdir

total 334M
drwxrwxr-x 2 olga olga 4.0K Sep 18 09:59 ./
drwxrwxr-x 3 olga olga 4.0K Sep 10 15:07 ../
-rw-rw-r-- 1 olga olga  52M Sep 18 07:37 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.bam
-rw-rw-r-- 1 olga olga 2.9M Sep 18 09:59 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.bam.bai
-rw-rw-r-- 1 olga olga 2.9M Sep 18 09:59 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.out_CDS_without_stop_codon_or_utr.bam
-rw-rw-r-- 1 olga olga  15M Sep 10 15:55 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codon.bam
-rw-rw-r-- 1 olga olga 1.7M Sep 18 07:05 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codon.bam.bai
-rw-rw-r-- 1 olga olga  15M Sep 18 07:15 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codon_or_utr.bam
-rw-rw-r-- 1 olga olga 1.7M Sep 18 07:06 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codon_or_utr.bam.bai

### Extract the read ids

In [112]:
! samtools view $outdir/SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.out_CDS_without_stop_codon_or_utr.bam | cut -f 1 > $outdir/SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.out_CDS_without_stop_codon_or_utr.read_ids.txt

In [115]:
! samtools view $outdir/SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.bam | cut -f 1 > $outdir/SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.read_ids.txt

In [113]:
! head $outdir/SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.out_CDS_without_stop_codon_or_utr.read_ids.txt

SRR306838.13598607
SRR306838.23303318
SRR306838.19917713
SRR306838.19937719
SRR306838.20862490
SRR306838.21924959
SRR306838.6854298
SRR306838.22584038
SRR306838.14605576
SRR306838.22585585


In [114]:
! wc -l $outdir/SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.out_CDS_without_stop_codon_or_utr.read_ids.txt

151869 /home/olga/data_sm/kmer-hashing/brawand2011/human/SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.out_CDS_without_stop_codon_or_utr.read_ids.txt


In [69]:
! samtools view $outdir/SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.out_CDS_without_stop_codon_or_utr.bam | head

[E::hts_hopen] Failed to open file /home/olga/data_sm/kmer-hashing/brawand2011/human//SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.out_CDS_without_stop_codon_or_utr.bam
[E::hts_open_format] Failed to open file /home/olga/data_sm/kmer-hashing/brawand2011/human//SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.out_CDS_without_stop_codon_or_utr.bam
samtools view: failed to open "/home/olga/data_sm/kmer-hashing/brawand2011/human//SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.out_CDS_without_stop_codon_or_utr.bam" for reading: Exec format error


In [ ]:
! samtools view $outdir/SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codon.bam 

In [40]:
outdir

'/home/olga/data_sm/kmer-hashing/brawand2011/human/'

In [41]:
ll $outdir

total 278872
-rw-rw-r-- 1 olga  15288774 Sep 10 15:55 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codon.bam
-rw-rw-r-- 1 olga  15288774 Sep 10 15:56 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codon_or_utr.bam
-rw-rw-r-- 1 olga   4287738 Sep 10 15:56 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codon_or_utr.read_ids.txt
-rw-rw-r-- 1 olga   4287738 Sep 10 15:56 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codon.read_ids.txt
-rw-r--r-- 1 olga 246383740 Sep 10 15:10 SRR306838_GSM752691_hsa_br_F_1_trimmed_k15.tsv


In [42]:
! samtools index $outdir/SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codon.bam

In [43]:
! samtools index $outdir/SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codon_or_utr.bam

In [68]:
ls -lha $outdir

total 331M
drwxrwxr-x 2 olga olga 4.0K Sep 18 07:37 ./
drwxrwxr-x 3 olga olga 4.0K Sep 10 15:07 ../
-rw-rw-r-- 1 olga olga  52M Sep 18 07:37 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.bam
-rw-rw-r-- 1 olga olga 2.9M Sep 18 09:54 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out.bam.unspliced.out_CDS_without_stop_codon_or_utr.bam
-rw-rw-r-- 1 olga olga  15M Sep 10 15:55 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codon.bam
-rw-rw-r-- 1 olga olga 1.7M Sep 18 07:05 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codon.bam.bai
-rw-rw-r-- 1 olga olga  15M Sep 18 07:15 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codon_or_utr.bam
-rw-rw-r-- 1 olga olga 1.7M Sep 18 07:06 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codon_or_utr.bam.bai
-rw-rw-r-- 1 olga olga 4.1M Sep 18 07:17 SRR306838_GSM752691_hsa_br_F_1Aligned.sortedByCoord.out_CDS_without_stop_codo

In [39]:
! bioawk -c gff '$start > 148292' $cds_no_stop_codons_or_utr_gtf | head

chr1	HAVANA	CDS	450743	451678	.	-	0	gene_id "ENSG00000284733.1"; transcript_id "ENST00000426406.3"; gene_type "protein_coding"; gene_name "OR4F29"; transcript_type "protein_coding"; transcript_name "OR4F29-201"; exon_number 1; exon_id "ENSE00002316283.3"; level 2; protein_id "ENSP00000409316.1"; transcript_support_level "NA"; tag "basic"; tag "appris_principal_1"; tag "CCDS"; ccdsid "CCDS72675.1"; havana_gene "OTTHUMG00000002860.3"; havana_transcript "OTTHUMT00000007999.3";
chr1	HAVANA	CDS	685719	686654	.	-	0	gene_id "ENSG00000284662.1"; transcript_id "ENST00000332831.4"; gene_type "protein_coding"; gene_name "OR4F16"; transcript_type "protein_coding"; transcript_name "OR4F16-201"; exon_number 1; exon_id "ENSE00002324228.3"; level 2; protein_id "ENSP00000329982.2"; transcript_support_level "NA"; tag "basic"; tag "appris_principal_1"; tag "CCDS"; ccdsid "CCDS41221.1"; havana_gene "OTTHUMG00000002581.3"; havana_transcript "OTTHUMT00000007334.3";
chr1	HAVANA	CDS	924432	924948	.	+	0	gene_i